In [1]:
%run setup.py

from scipy.optimize import curve_fit
from grid import match_modes

# Condense the model grid into numpy binary objects

In [ ]:
atm = 'hopf' # 'eddin'
input_dir = '../../hpc/coarse_v2_{}/'.format(atm) # the directory that stores orignial stellar models
grids = pd.read_csv(input_dir+'template/coarse_grid_input_params_v2.txt') 

In [ ]:
from tqdm import tqdm
import astropy.table
from grid import get_model_Dnu
import scipy.interpolate

for i in range(0, 9): #

    start = i*1000
    end = (i+1)*1000-1 if (i+1)*1000-1 < (2**13) else 2**13
    sl = slice(start, end, 1)
    print(sl)
    
    
    def multi(inputs):
        it, track = inputs
        filepath = input_dir+'pre_rgb_tip/complete_grid/index{:06.0f}.history.h5'.format(track['index'])
        if not os.path.exists(filepath): 
            return None

        cols = ['index', 'profile_number', 'flag_seismo', 
                'Yinit', 'Xinit', 'Zinit', 'fov_core', 'fov_shell', 'amlt', 
                'star_age', 'star_mass', 'luminosity', 'radius', 'Teff', 
                'log_g', 'FeH', 'delta_nu_scaling', 'numax_scaling', 'acoustic_cutoff']

        f = h5py.File(filepath, 'r')
        # print(filepath)
        cols = ['index', 'profile_number', 'flag_seismo', 
                'star_age', 'star_mass', 'luminosity', 'radius', 'Teff', 'delta_nu_scaling', 'numax_scaling', 'acoustic_cutoff', 'delta_Pg', 'log_g',
                'Yinit', 'Xinit', 'Zinit', 'center_h1', 'fov_core', 'fov_shell', 'amlt', 'FeH', 'log_LHe', 'log_Lnuc']


        atrack = Table([f[col][:] for col in cols], names=cols)
        idx = (atrack['flag_seismo']==1) & (atrack['numax_scaling']>15) & ((atrack['log_LHe']-atrack['log_Lnuc'])<-3)

        atrack = atrack[idx]
        atrack['density'] = atrack['star_mass']**0.5/atrack['radius']**-1.5
        atrack['numax'] = atrack['numax_scaling']
        atrack['Dnu'] = atrack['delta_nu_scaling']

        atrack['[M/H]'] = atrack['FeH']
        atrack['mode_l'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_n'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_freq_o'] = np.zeros(len(atrack), dtype=object)
        # atrack['mode_freq'] = np.zeros(len(atrack), dtype=object)
        atrack['mode_inertia'] = np.zeros(len(atrack), dtype=object)
        
        # # # replace the acoustic_cutoff from mesa with numax_scaling
        atrack['acoustic_cutoff'] = atrack['numax_scaling']
        atrack['Dnu_freq_o'] = np.zeros(len(atrack))
        atrack['eps_o'] = np.zeros(len(atrack))
        atrack['surf_corr_at_numax'] = np.zeros(len(atrack))
        # #
        for i in range(len(atrack)):
            # historyIndex = atrack[i]['index']
            profileIndex = atrack[i]['profile_number']

            numax_scaling = atrack[i]['numax_scaling']
            g = atrack[i]['star_mass']/atrack[i]['radius']**2.0
            Teff = atrack[i]['Teff']
            feh = atrack[i]['FeH']
            
            atrack['mode_l'][i] = f['profile{:0.0f}/{:s}'.format(profileIndex, 'l')][:][0]
            atrack['mode_n'][i] = f['profile{:0.0f}/{:s}'.format(profileIndex, 'n_pg')][:][0]
            mode_freq = f['profile{:0.0f}/{:s}'.format(profileIndex, 'freq')][:][0]
            mode_inertia = f['profile{:0.0f}/{:s}'.format(profileIndex, 'E_norm')][:][0]
            
            atrack['Dnu_freq_o'][i], atrack['eps_o'][i] = get_model_Dnu(mode_freq, atrack['mode_l'][i], \
                                                    atrack['delta_nu_scaling'][i], atrack['numax_scaling'][i], atrack['mode_n'][i])
            
            # thetas = np.array([-6.185296 ,  0.7798102, -4.807426 , -0.7790131, 
            #                    -7.7618823,  0.7829924, -4.3983994, -0.8528419])
                
            # D1 = thetas[0] * (g)**thetas[1] * (Teff/5777.)**thetas[2] * (thetas[3]*feh + 1) # surf corr at numax
            # D2 = thetas[4] * (g)**thetas[5] * (Teff/5777.)**thetas[6] * (thetas[7]*feh + 1) # surf corr at scale*numax
            # scale = 1.1

            # fi = scipy.interpolate.interp1d(mode_freq, (mode_freq/numax_scaling)**3.0/mode_inertia, kind='cubic')
            # gs = fi(numax_scaling)

            # fi = scipy.interpolate.interp1d(mode_freq, (mode_freq/numax_scaling)**-1.0/mode_inertia, kind='cubic')
            # fs = fi(numax_scaling)

            # surface_a3 = (scale**-1 * D1 - D2)/ ((scale**-1 - scale**3) *gs)
            # surface_a1 = (scale**3 * D1 - D2)/ ((scale**3 - scale**-1) *fs)
            # dfreq = (surface_a3 *(mode_freq/numax_scaling)**3.0+surface_a1 *(mode_freq/numax_scaling)**-1.0)/mode_inertia
            
            atrack['mode_freq_o'][i] = mode_freq
            # atrack['mode_freq'][i] = mode_freq + dfreq
            atrack['mode_inertia'][i] = mode_inertia
            # atrack['surf_corr_at_numax'][i] = D1
            
        return atrack
    

    tracks = process_map(multi, grids.loc[sl,:].iterrows(), total=1000, max_workers=12)
    data = astropy.table.vstack([track for track in tracks if track is not None])
    np.save('tracks_{:0.0f}_{:0.0f}'.format(sl.start, sl.stop), data)
